In [2]:
!pip install gdown

In [3]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
from PIL import Image,ImageOps
warnings.filterwarnings(action='ignore') 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
!pip install gdown
!gdown --id 1uo351ez10bN24Xbs_kpfwqCSn3qiL0cn

In [ ]:
!unzip /kaggle/working/Images.zip

In [4]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':10,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':16,
    'SEED':42
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
df=pd.read_csv('/kaggle/input/facescore-labels/All_labels.txt',sep=' ',header=None)
df.columns=['img_path','label']

In [7]:
def change_path(path):
    path=path.replace(path,'/kaggle/working/'+path)
    return path
df['img_path']=df['img_path'].apply(change_path)

In [8]:
train,val=train_test_split(df,test_size=0.2,random_state=CFG['SEED'])

In [9]:
train

,img_path,label
4315,/kaggle/working/AM1914.jpg,3.383333
1832,/kaggle/working/CF376.jpg,2.450000
3819,/kaggle/working/CM191.jpg,2.983333
957,/kaggle/working/AM1298.jpg,2.616667
3545,/kaggle/working/AF966.jpg,3.033333
...,...,...
3772,/kaggle/working/AF1082.jpg,2.633333
5191,/kaggle/working/AM1877.jpg,4.033333
5226,/kaggle/working/CF206.jpg,2.133333
5390,/kaggle/working/AM959.jpg,2.083333


In [10]:
class CustomDataset(Dataset):
    def __init__(self,imgs_path,labels,transforms=None):
        self.imgs_path=imgs_path
        self.labels=labels
        self.transforms=transforms
    def __getitem__(self,index):
        img_path=self.imgs_path[index]
        image=cv2.imread(img_path)
        if self.transforms is not None:
            image=self.transforms(image=image)['image']
        if self.labels is not None:
            label=self.labels[index]
            return image,label
        else:
            return image
    def __len__(self):
        return len(self.imgs_path)
train_transform=A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE'],interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2()
])
test_transform=A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE'],interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2()
])
train_dataset=CustomDataset(train['img_path'].values,train['label'].values,train_transform)
val_dataset=CustomDataset(val['img_path'].values,val['label'].values,test_transform)
train_loader=DataLoader(train_dataset,batch_size=CFG['BATCH_SIZE'],shuffle=True,num_workers=0)
val_loader=DataLoader(val_dataset,batch_size=CFG['BATCH_SIZE'],shuffle=False,num_workers=0)

In [11]:
import timm
model=timm.create_model('timm/swinv2_large_window12to16_192to256.ms_in22k_ft_in1k', pretrained=True,num_classes=1)
model = torch.nn.DataParallel(model)

In [18]:
from sklearn.metrics import mean_absolute_error
import copy
def train(model, optimizer, train_loader, val_loader, scheduler, device, early_stopping_patience=3):
    criterion = nn.MSELoss().to(device)
    model.to(device)
    best_model = None
    best_loss = float('inf')
    patience = 0

    for epoch in range(1,CFG['EPOCHS']+1):
        train_loss = []
        train_preds, train_score_true = [], []
        model.train()

        for img, score in tqdm(iter(train_loader)):
            img = img.float().to(device)
            score = score.float().to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, score) 
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            train_preds += output.detach().cpu().numpy().tolist()  
            train_score_true += score.detach().cpu().numpy().tolist()
        _train_loss=np.mean(train_loss)
        _train_mae = mean_absolute_error(train_score_true, train_preds)
        _val_mae, _val_loss= validation(model, val_loader, criterion, device)
        current_lr = optimizer.param_groups[0]['lr']

        print(f'Epoch [{epoch}], Train MAE: [{_train_mae:.5f}], Train Loss: [{_train_loss:.5f}], Val MAE: [{_val_mae:.5f}], Val Loss: [{_val_loss:.5f}], Learning Rate: {current_lr}')

        if scheduler is not None:
            scheduler.step()

        if best_loss > _val_mae:
            best_loss = _val_mae
            best_model = copy.deepcopy(model)
            torch.save(best_model, 'swinv2_age.pt')
            patience = 0
        else:
            patience += 1

        if patience >= early_stopping_patience:
            print(f'Early stopping triggered at epoch {epoch}!')
            break

    return best_model

def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    val_preds, val_score_true = [], []

    with torch.no_grad():
        for img, score in tqdm(iter(val_loader)):
            img = img.float().to(device)
            score= score.float().to(device)
            pred = model(img)
            loss = criterion(pred, score)  # pred를 squeeze하여 차원을 맞춤

            val_preds += pred.detach().cpu().numpy().tolist()  # flatten을 사용하여 1차원 배열로 변환
            val_score_true += score.detach().cpu().numpy().tolist()
            val_loss.append(loss.item())
        _val_loss=np.mean(val_loss)
        _val_mae = mean_absolute_error(val_score_true, val_preds)

    return _val_mae, _val_loss

In [19]:
model.eval()
optimizer=torch.optim.AdamW(params=model.parameters(),lr=CFG['LEARNING_RATE'])
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=10)
infer_model=train(model,optimizer,train_loader,val_loader,scheduler,device)

100%|██████████| 69/69 [00:33<00:00,  2.04it/s]


Epoch [1], Train MAE: [0.55607], Train Loss: [0.48700], Val MAE: [0.58232], Val Loss: [0.49086], Learning Rate: 1e-05


100%|██████████| 69/69 [00:33<00:00,  2.04it/s]


Epoch [2], Train MAE: [0.55519], Train Loss: [0.49135], Val MAE: [0.54555], Val Loss: [0.47732], Learning Rate: 9.755282581475769e-06


100%|██████████| 69/69 [00:33<00:00,  2.04it/s]


Epoch [3], Train MAE: [0.55324], Train Loss: [0.48348], Val MAE: [0.56918], Val Loss: [0.47537], Learning Rate: 9.045084971874738e-06


100%|██████████| 69/69 [00:33<00:00,  2.04it/s]


Epoch [4], Train MAE: [0.54853], Train Loss: [0.48206], Val MAE: [0.54670], Val Loss: [0.48074], Learning Rate: 7.938926261462366e-06


100%|██████████| 69/69 [00:33<00:00,  2.05it/s]

Epoch [5], Train MAE: [0.54710], Train Loss: [0.47955], Val MAE: [0.55262], Val Loss: [0.47409], Learning Rate: 6.545084971874738e-06
Early stopping triggered at epoch 5!


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [22]:
import os
for i in df['img_path']:
    os.remove(i)